In [1]:
1

1

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import jax
jax.config.update("jax_enable_x64", True)

In [4]:
import scvi
import scanpy as sc
import os
import pandas as pd
import numpy as np
from cfp.external import CFJaxSCVI

/home/icb/dominik.klein/mambaforge/envs/cfp/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
split = 5
output_dir = "/lustre/groups/ml01/workspace/ot_perturbation/data/sciplex"
adata_train = sc.read(os.path.join(output_dir, f"adata_train_{split}.h5ad"))
adata_test = sc.read(os.path.join(output_dir, f"adata_test_{split}.h5ad"))
adata_ood = sc.read(os.path.join(output_dir, f"adata_ood_{split}.h5ad")) 




In [7]:
vae = CFJaxSCVI.load("/lustre/groups/ml01/workspace/ot_perturbation/models/vaes/sciplex/32_1024", adata=adata_train)

INFO     File /lustre/groups/ml01/workspace/ot_perturbation/models/vaes/sciplex/32_1024/model.pt already downloaded


/home/icb/dominik.klein/mambaforge/envs/cfp/lib/python3.11/site-packages/lightning/fabric/plugins/environments/slurm.py:191: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /home/icb/dominik.klein/mambaforge/envs/cfp/lib/pyth ...
/home/icb/dominik.klein/mambaforge/envs/cfp/lib/python3.11/site-packages/scvi/model/base/_utils.py:66: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be a

INFO     Jax module moved to cuda:0.Note: Pytorch lightning will show GPU is not being used for the Trainer.       


/home/icb/dominik.klein/mambaforge/envs/cfp/lib/python3.11/site-packages/lightning/fabric/plugins/environments/slurm.py:191: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /home/icb/dominik.klein/mambaforge/envs/cfp/lib/pyth ...
/home/icb/dominik.klein/mambaforge/envs/cfp/lib/python3.11/site-packages/lightning/fabric/plugins/environments/slurm.py:191: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /home/icb/dominik.klein/mambaforge/envs/cfp/lib/pyth ...
GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/icb/dominik.klein/mambaforge/envs/cfp/lib/python3.11/site-packages/lightning/pytorch/trainer/setup.py:187: GPU availabl

Epoch 1/18:   0%|          | 0/18 [00:00<?, ?it/s]

/home/icb/dominik.klein/mambaforge/envs/cfp/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:385: You have overridden `transfer_batch_to_device` in `LightningModule` but have passed in a `LightningDataModule`. It will use the implementation from `LightningModule` instance.


Epoch 1/18:   6%|▌         | 1/18 [00:08<02:20,  8.29s/it, v_num=1, train_loss_step=402, train_loss_epoch=402]

`Trainer.fit` stopped: `max_steps=1` reached.


Epoch 1/18:   6%|▌         | 1/18 [00:08<02:22,  8.38s/it, v_num=1, train_loss_step=402, train_loss_epoch=402]


In [8]:
adata_train.obsm["X_scVI"] = vae.get_latent_representation(adata_train)
adata_test.obsm["X_scVI"] = vae.get_latent_representation(adata_test)
adata_ood.obsm["X_scVI"] = vae.get_latent_representation(adata_ood)



INFO     Input AnnData not setup with scvi-tools. attempting to transfer AnnData setup                             


/home/icb/dominik.klein/mambaforge/envs/cfp/lib/python3.11/site-packages/scvi/data/fields/_base_field.py:64: UserWarning: adata.X does not contain unnormalized count data. Are you sure this is what you want?
  self.validate_field(adata)


INFO     Input AnnData not setup with scvi-tools. attempting to transfer AnnData setup                             


/home/icb/dominik.klein/mambaforge/envs/cfp/lib/python3.11/site-packages/scvi/data/fields/_base_field.py:64: UserWarning: adata.X does not contain unnormalized count data. Are you sure this is what you want?
  self.validate_field(adata)


In [9]:
from cfp.training import VAEDecodedMetrics
import cfp

In [10]:
adata_train.obs.columns

Index(['cell_type', 'dose', 'dose_character', 'dose_pattern', 'g1s_score',
       'g2m_score', 'pathway', 'pathway_level_1', 'pathway_level_2',
       'product_dose', 'product_name', 'proliferation_index', 'replicate',
       'size_factor', 'target', 'vehicle', 'perturbation', 'drug', 'cell_line',
       'logdose', 'condition', 'n_genes', 'pubchem_name', 'pubchem_ID',
       'smiles', 'control', 'ood_1', 'ood_2', 'ood_3', 'ood_4', 'ood_5',
       'split', '_scvi_batch'],
      dtype='object')

In [11]:
cf = cfp.model.CellFlow(adata_train, solver="otfm")

cf.prepare_data(
    sample_rep="X_scVI",
    control_key="control",
    perturbation_covariates= {"drugs": ["drug"], "dosage": ["logdose"]},
    perturbation_covariate_reps={"drugs": "ecfp_dict"},
    sample_covariates=["cell_line"],
    sample_covariate_reps={"cell_line": 'cell_line_dict'},
    split_covariates=["cell_line"],
)




/ictstr01/home/icb/dominik.klein/git_repos/cell_flow_perturbation/src/cfp/data/_datamanager.py:334: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  _covariate_data["cell_index"] = _covariate_data.index
100%|██████████| 660/660 [00:11<00:00, 58.47it/s]


In [12]:
import functools
import optax
from ott.solvers import utils as solver_utils
match_fn = functools.partial(
    solver_utils.match_linear,
    epsilon=1.0,
    scale_cost="mean",
)
optimizer = optax.MultiSteps(optax.adam(1e-4), 20)

# Prepare the model
cf.prepare_model(
    encode_conditions=True,
    condition_embedding_dim=1024,
    pooling="attention_token",
    )


In [13]:
#cf.prepare_validation_data(
#    adata_test,
#    name="test",
#    n_conditions_on_log_iteration=2,
#    n_conditions_on_train_end=2,
#)

cf.prepare_validation_data(
    adata_ood,
    name="ood",
    n_conditions_on_log_iteration=2,
    n_conditions_on_train_end=2,
)


/ictstr01/home/icb/dominik.klein/git_repos/cell_flow_perturbation/src/cfp/data/_datamanager.py:334: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  _covariate_data["cell_index"] = _covariate_data.index
100%|██████████| 25/25 [00:00<00:00, 417.20it/s]


In [14]:
metrics_callback = cfp.training.Metrics(metrics=["r_squared", "mmd", "e_distance"])
decoded_metrics_callback = cfp.training.VAEDecodedMetrics(vae, adata=adata_train, metrics=["r_squared"])
wandb_callback = cfp.training.WandbLogger(project="cfp_otfm_sciplex", out_dir="/home/icb/dominik.klein/tmp", config={})


In [15]:
callbacks = [metrics_callback, decoded_metrics_callback, wandb_callback]

In [ ]:
cf.train(
    num_iterations=1000,
    batch_size=1024,
    callbacks=callbacks,
    valid_freq=20,
)
   

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: mucdk (modality_translation). Use `wandb login --relogin` to force relogin


  2%|▏         | 18/1000 [00:21<02:51,  5.73it/s] 

data is  [[ 1.2531824e-02  1.1047505e+00 -9.8844886e-01 ...  4.1933823e-03
  -5.1874681e-03 -1.7095216e-03]
 [-8.8377483e-03  2.1481929e+00  1.3905278e-01 ... -1.3914330e-02
  -1.8148536e-02 -2.4921631e-03]
 [-2.3707479e-02  1.3324978e+00 -1.2112579e-01 ... -1.1239069e-02
   4.8553204e-04  1.2016719e-02]
 ...
 [-1.2723635e-02  1.4950398e+00 -7.1578991e-01 ... -7.0440369e-03
   1.9683372e-03  1.7811233e-02]
 [-1.0520482e-02  2.2472453e+00  2.6795559e+00 ...  1.7473670e-03
   1.3639921e-02  8.9097861e-03]
 [-9.3352413e-03  1.4724388e+00 -1.7890689e-01 ... -6.9763046e-03
  -5.9305574e-03  1.1611126e-02]]
data is  [[ 2.9154364e-03 -4.2029750e-01 -6.2267864e-01 ...  1.2524012e-03
  -3.2466378e-03  2.9241201e-05]
 [-4.1480120e-03 -8.8686693e-01 -1.3223681e+00 ... -3.7375072e-03
  -1.7004258e-03  7.7369809e-04]
 [ 5.1181521e-03 -4.2105746e-01 -8.1826699e-01 ...  2.2320142e-03
  -6.4227474e-03 -8.0334139e-04]
 ...
 [-1.6588010e-03 -1.7779232e+00 -5.3553391e-01 ...  1.5321407e-03
   5.4185148e-

/home/icb/dominik.klein/mambaforge/envs/cfp/lib/python3.11/site-packages/scvi/data/fields/_base_field.py:64: UserWarning: adata.X does not contain unnormalized count data. Are you sure this is what you want?
  self.validate_field(adata)


INFO     Input AnnData not setup with scvi-tools. attempting to transfer AnnData setup                             


...

/home/icb/dominik.klein/mambaforge/envs/cfp/lib/python3.11/site-packages/scvi/data/fields/_base_field.py:64: UserWarning: adata.X does not contain unnormalized count data. Are you sure this is what you want?
  self.validate_field(adata)


INFO     Input AnnData not setup with scvi-tools. attempting to transfer AnnData setup                             


/home/icb/dominik.klein/mambaforge/envs/cfp/lib/python3.11/site-packages/scvi/data/fields/_base_field.py:64: UserWarning: adata.X does not contain unnormalized count data. Are you sure this is what you want?
  self.validate_field(adata)


INFO     Input AnnData not setup with scvi-tools. attempting to transfer AnnData setup                             
INFO     Input AnnData not setup with scvi-tools. attempting to transfer AnnData setup                             


/home/icb/dominik.klein/mambaforge/envs/cfp/lib/python3.11/site-packages/scvi/data/fields/_base_field.py:64: UserWarning: adata.X does not contain unnormalized count data. Are you sure this is what you want?
  self.validate_field(adata)
/home/icb/dominik.klein/mambaforge/envs/cfp/lib/python3.11/site-packages/scvi/data/fields/_base_field.py:64: UserWarning: adata.X does not contain unnormalized count data. Are you sure this is what you want?
  self.validate_field(adata)
/home/icb/dominik.klein/mambaforge/envs/cfp/lib/python3.11/site-packages/scvi/data/_anntorchdataset.py:151: RuntimeWarning: overflow encountered in cast
  sliced_data = data[indexes].astype(dtype, copy=False)
/home/icb/dominik.klein/mambaforge/envs/cfp/lib/python3.11/site-packages/numpy/core/_methods.py:49: RuntimeWarning: invalid value encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims, initial, where)
  6%|▌         | 57/1000 [00:41<09:02,  1.74it/s, loss=0.075]